In [5]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from scipy.spatial import distance_matrix

from src.gnn import GNNClassifier
from utils.utils import get_adjacency_and_features

import sys
import numpy as np
sys.path.append('../datasets')
from datasets.manager import IMDBBinary, DD
import torch 

In [6]:
IMDB = IMDBBinary()
DD = DD()

In [7]:
x_IMBD, y_IMDB = IMDB.dataset.get_data(), IMDB.dataset.get_targets()

In [9]:
node8 = x_IMBD[8]
adjency_mat, feature_vect = get_adjacency_and_features(node8)

In [10]:
model = GNNClassifier(in_features=1, 
                      embedding_dim=1, 
                      out_features=1, 
                      num_layers=3)

out = model(feature_vect, adjency_mat)

out 

tensor([-0.5827,  0.2425], grad_fn=<ViewBackward0>)

In [6]:
# Routine d'entraînement pour la classification de graphes
def train_gnn(model, dataset, epochs=10, lr=0.001, batch_size=32):
    """Entraîne un modèle GNN sur le dataset de graphes MNIST."""
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: list(zip(*x)))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        total_loss = 0
        for graphs, labels in dataloader:
            labels = torch.tensor(labels, dtype=torch.long)
            optimizer.zero_grad()
            output = model(graphs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")